<a href="https://colab.research.google.com/github/HimashiRathnayake/Hate-Speech-Humor-Detection/blob/main/XLM-R/SelectiveTranslateTransliterate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Selective Translation and Transliteration (STT)**

**Dependencies**

In [ ]:
!pip install googletrans==4.0.0rc1
!pip install django
!pip install langid
!pip install ai4bharat-transliteration
!pip install inflection

     |████████████████████████████████| 55 kB 2.8 MB/s 
     |████████████████████████████████| 42 kB 1.0 MB/s 
     |████████████████████████████████| 1.2 MB 28.1 MB/s 
     |████████████████████████████████| 65 kB 3.1 MB/s 
     |████████████████████████████████| 53 kB 1.9 MB/s 
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17415 sha256=f156241091c14a5bd364a3ff0cf92e52fcafefe368a24bac715947c6937fba83
  Stored in directory: /root/.cache/pip/wheels/43/34/00/4fe71786ea6d12314b29037620c36d857e5d104ac2748bf82a
Successfully built googletrans
     |████████████████████████████████| 7.9 MB 10.9 MB/s 
     |████████████████████████████████| 1.9 MB 13.2 MB/s 
  Created wheel for langid: filename=langid-1.1.6-py3-none-any.whl size=1941187 sha256=557061e714e32860abc80cc44e4b87027d3fdf2c8ce78b221f45a0c782b4dd1b
  Stored in directory: /root/.cache/pip/wheels/2b/bb/7f/11e4db39477278161e882eadc46fb558949a28b13470fc74b8
Successfully built langid


In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
from google.colab import drive
from googletrans import Translator
from urllib.request import urlopen
import django.utils.encoding
from django.utils import http
import langid
from nltk.corpus import words
import nltk
import string
from ai4bharat.transliteration import XlitEngine
import inflection

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


**Apply TPU Support**

In [ ]:
print("Tensorflow version " + tf.__version__)

try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
# tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

**Load Data**

In [ ]:
dataset_path = "/content/drive/Shareddrives/FYP/corpus/çompleted_draft.csv"
all_data = pd.read_csv(dataset_path)

**Selective Translation and Transliteration**

In [ ]:
nltk.download('words')
nltk.download('punkt')

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
tokenizer = nltk.RegexpTokenizer(r"\w+")
transliterator = XlitEngine("si")

Loading si...


In [ ]:
def selective_translation_transliteration(input_text, output_script="si", variation=2):
  print(input_text)
  output_text = input_text
  tokens = tokenizer.tokenize(input_text)
  for i, token in enumerate(tokens):

    ## googletrans to detect language -  Gives a request limit error
    # translator = Translator()
    # lang = translator.detect(token).lang

    ## Langid api
    lang = langid.classify(token)[0]
    if (lang == output_script):
      continue
    elif (inflection.singularize(token.lower()) in words.words()):
      if (variation==1):
        translator = Translator()
        output_text = output_text.replace(token, translator.translate(token, dest=output_script).text, 1)
      else:
        continue
    else:
      try:
        # url=u' http://www.cfilt.iitb.ac.in/indicnlpweb/indicnlpws/transliterate_bulk/en/si/{}/rule'.format(token.lower())
        # response = urlopen(url).read()
        # response = django.utils.encoding.uri_to_iri(response)
        # output_text = output_text.replace(token, response.split('{"si":["')[1].split('"]}')[0], 1)
        transliterated_token = list(transliterator.translit_word(token).values())[0][0]
        output_text = output_text.replace(token, transliterated_token, 1)
      except UnicodeEncodeError:
        continue
  print(output_text)
  return output_text

In [ ]:
print(all_data.Sentence[0])

# all_data['Sentence'] = selective_translation_transliteration(all_data['Sentence'].to_numpy())
# df['new_col'] = multiply_numba(df['a'].to_numpy())
# print('\n', all_data)
# selective_translation_transliteration(all_data.Sentence[1])

from multiprocessing import Pool

def apply_translation_transliteration(df):
    return df['Sentence'].apply(selective_translation_transliteration)

n_cores = 8
pool = Pool(n_cores)

def apply_parallel(df, func):
    # split dataframe
    df_split = np.array_split(all_data, n_cores)
    # calculate metrics for each and concatenate
    df = pd.concat(pool.map(func, df_split))
    return df

all_data = all_data[:5000]

all_data['Sentence'] = apply_parallel(all_data, apply_translation_transliteration)

all_data.to_csv("/content/drive/Shareddrives/FYP/corpus/çompleted_draft_translate_transliterate.csv")

Ammage Adarayta❤️Eka Dawasak Madi Neda❤️🙏❤️
Ammage Adarayta❤️Eka Dawasak Madi Neda❤️🙏❤️
අම්මාගේ ආදරයට❤️එක දවසක් මදි නේද❤️🙏❤️
We need IPL Champions leak data offers ..please provide it..last year also you all provided 😊
We need ඉප් Champions leak data offers ..please provide it..last year also you all provided 😊
#VPN #ummmaaa #proud_be
#විපින් #උම්මා #ප්‍රෝඩ්_බෙ
chandimal.. uuu thama mulu tem ekama kaaa gahala hari quick karala ganne
චන්දිමාල්.. ඌරු තමා මුලු තෙම් එකම කා ගහල හරී quick කරල ගන්නේ
sltgo
ස්ල්ටෝ
Dialog 49k dammama eka dawasak wath be
ඩයලොග් 49ක් ධම්මාම එක දවසක් wath be
eth anith kattiya sathiyak withara online innawa 49 data eken..
ඒත් අනිත් කත්තිය සතියක් විතර ඔන්ලයින් ඉන්නව 49 data එකෙන්..
meka salli kanawane...ai e?
මේක සල්ලි කනවනේ...ai e?
Navy🧡❤️💛💗💚💜️
Navy🧡❤️💛💗💚💜️
One ring to bomb them all .Terrorist style
One ring to bomb them all .Terrorist style
Answer no 1) 45 1st president - Jeorge Washington 2nd " - John Adams 3rd " - Thomas Jefferson . . . 44th " - Barack Obama 45th

Process ForkPoolWorker-116:
Traceback (most recent call last):
  File "/usr/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
  File "/usr/lib/python3.7/multiprocessing/pool.py", line 44, in mapstar
    return list(map(*args))
  File "<ipython-input-95-216a97cae0cd>", line 11, in apply_translation_transliteration
    return df['Sentence'].apply(selective_translation_transliteration)
  File "/usr/local/lib/python3.7/dist-packages/pandas/core/series.py", line 4213, in apply
    mapped = lib.map_infer(values, f, convert=convert_dtype)
  File "pandas/_libs/lib.pyx", line 2403, in pandas._libs.lib.map_infer
  File "<ipython-input-94-eaa880e500ed>", line 23, in selective_translation_transliteration
    transliterated_token = list(tr

KeyboardInterrupt: ignored

In [ ]:
all_data.to_csv("/content/drive/Shareddrives/FYP/corpus/çompleted_draft_translate_transliterate_1.csv")

In [ ]:
all_data

In [ ]:
import re

def preprocess_texts(sentences):
  sentences = [re.sub(r'http\S+','',s) for s in sentences]
  sentences = [s.replace('#','') for s in sentences]
  return sentences

preprocess_texts(["https://drive.google.com/drive/folders/1OkGF9EQ7FmOI597OwcOzhNFLOG6e2oD8 here #yes"])

[' here yes']